<a href="https://colab.research.google.com/github/0jipy/192kbps_colab_JIPY/blob/main/%EC%87%BC%ED%95%91%EC%95%A4%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0%EC%84%9D%EC%96%B4%20%EA%B0%9C%EB%B0%9C%ED%95%B4%EC%95%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 네이버 쇼핑 리뷰와, 영화평 리뷰를 섞은 데이터
# - vocab_size, max_len, embedding_vector, hidden_units
# - SimpleRNN, GRU, LSTM, Conv1D

In [ ]:
'''!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220111.sh'''

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220111.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 4.39 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 69.1 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-01-28 00:28:39--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::22c5:2ef4, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loca

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt'
df1 = pd.read_table(url, names=['ratings', 'reviews'])
df1.head()

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [ ]:
# 평점이 4, 5점인 데이터를 1(긍정)
df1['label'] = df1.apply(lambda x:1 if x.ratings > 3 else 0, axis=1)
df1.head()

,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [ ]:
# 각 열별로 중복 확인
df1.shape, df1.ratings.nunique(), df1.reviews.nunique()

((200000, 3), 4, 199908)

In [ ]:
# 중복 제거
df1.drop_duplicates(subset=['reviews'], inplace=True)

In [ ]:
# Null 값 제거 - 0 이 나오면 결측치가 없음
df1.isnull().sum().sum()

0

In [ ]:
df1.reviews = df1.reviews.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')

In [ ]:
# Null 데이터가 생기면 제거
df1.reviews.replace('', np.nan, inplace=True)
df1.reviews.isnull().sum()

0

In [ ]:
df2 = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", sep='\t')
df2.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
# 중복 여부 확인
df2.shape, df2.document.nunique()

((200000, 3), 194543)

In [ ]:
# 중복 데이터 배제
df2.drop_duplicates(subset=['document'], inplace=True)
df2.shape

(194544, 3)

In [ ]:
# Null 데이터 있는지 확인
df2.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [ ]:
# Null data 제거
df2.dropna(how='any', inplace=True)

In [ ]:
# # 한글 이외의 문자는 공백으로 처리하고 strip
# df2.document = df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
# df2.head(3)

NameError: ignored

In [ ]:
# '' 만 남은 데이터 --> np.nan으로 변경시킨 후 제거
df2.document.replace('', np.nan, inplace=True)
df2.document.isna().sum()

0

In [ ]:
# Null data 제거
df2.dropna(how='any', inplace=True)
df2.shape

(194543, 3)

In [ ]:
df1.rename(columns={'reviews':'document'}, inplace=True)
df1.head(3)

,ratings,document,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서개 더 구매했어요 이가격에 대박입니다 바느질이 조금 엉성...,1


In [ ]:
df1 = df1[['document', 'label']]
df1.head(3)

,document,label
0,배공빠르고 굿,1
1,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,아주좋아요 바지 정말 좋아서개 더 구매했어요 이가격에 대박입니다 바느질이 조금 엉성...,1


In [ ]:
df2 = df2[['document', 'label']]
df2.head(3)

,document,label
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1


In [ ]:
df = pd.merge(df1, df2, how='outer')
df.head()

,document,label
0,배공빠르고 굿,1
1,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,아주좋아요 바지 정말 좋아서개 더 구매했어요 이가격에 대박입니다 바느질이 조금 엉성...,1
3,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...,0
4,민트색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [ ]:
# 중복데이터 확인
df.shape ,df.document.nunique()

((394403, 2), 393866)

In [ ]:
# 중복 데이터 배제
df.drop_duplicates(subset=['document'], inplace=True)
df.shape

(393866, 2)

In [ ]:
# 훈련 데이터 : 테스트 데이터 = 3 : 1
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.document.values, df.label.values, stratify=df.label.values, 
    test_size = 0.25, random_state=2022
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((295399,), (98467,), (295399,), (98467,))

In [ ]:
# 훈련 데이터 레이블의 분포
np.unique(y_train, return_counts=True)

(array([0, 1]), array([147652, 147747]))

In [ ]:
from konlpy.tag import Mecab

In [ ]:
mecab = Mecab()
stopwords = stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']


In [ ]:
X_train = [mecab.morphs(review) for review in X_train]
X_train = list(map(lambda s: [item for item in s if item not in stopwords], X_train))

In [ ]:
X_test = [mecab.morphs(review) for review in X_test]
X_test = list(map(lambda s: [item for item in s if item not in stopwords], X_test))